<a href="https://colab.research.google.com/github/sg5993/AI_DC_GALLERY_BOT_/blob/main/AI%20%EB%94%94%EC%8B%9C%20%EA%B0%A4%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <-- 모바일 튕김 방지용 { display-mode: "form" }
%%html
<b>음악 재생으로 콜랩 활동 유지 시키기</b><br/>
<audio autoplay="" src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" loop controls>

In [ ]:
pip install dc_api filetype

In [ ]:

import asyncio
import dc_api
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import random
from time import sleep, time
import os
from collections import Counter
from datetime import datetime
import lxml.html
import yaml
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Google API 키 설정
GOOGLE_API_KEY = 'Gemini api 넣기'  # 실제 Google API 키로 변경
genai.configure(api_key=GOOGLE_API_KEY)

# Gemini 모델 설정
model = genai.GenerativeModel(model_name='gemini-1.5-flash')
generation_config = genai.GenerationConfig(
    temperature=0.6,
    top_k=1,
    max_output_tokens=750
)

# 디시 봇 클래스
class DcinsideBot:
    def __init__(self, board_id, username, password, persona, memory_path, memory_file,
                 max_run_time, comment_interval, crawl_article_count,
                 comment_target_count, write_article_enabled, write_comment_enabled,
                 record_memory_enabled, record_data_enabled, article_interval,
                 use_time_limit, load_memory_enabled, load_data_enabled):
        self.board_id = board_id
        self.username = username
        self.password = password
        self.persona = persona
        self.api = None  # dc_api.API() 객체는 run_gallery_bot 함수에서 설정
        self.gallery_name = None  # 갤러리 이름 저장 변수
        self.memory_file = memory_file
        self.memory_path = memory_path
        self.max_run_time = max_run_time
        self.comment_interval = comment_interval  # 댓글 작성 간격
        self.crawl_article_count = crawl_article_count  # 크롤링할 글 개수
        self.comment_target_count = comment_target_count  # 댓글 대상 글 개수

        self.write_article_enabled = write_article_enabled
        self.write_comment_enabled = write_comment_enabled
        self.record_memory_enabled = record_memory_enabled
        self.record_data_enabled = record_data_enabled
        self.article_interval = article_interval  # 글 작성 간격 (초)
        self.use_time_limit = use_time_limit  # 시간 제한 사용 여부
        self.load_memory_enabled = load_memory_enabled  # 갤러리 기억 파일 로드 활성화 여부
        self.load_data_enabled = load_data_enabled  # 데이터 파일 로드 활성화 여부

        self.last_comment_time = 0  # 마지막 댓글 작성 시간 초기화
        self.last_document_time = 0  # 마지막 글 작성 시간 초기화

    async def write_article(self, trending_topics, memory_data=None):
        """언어 모델을 이용하여 글 제목과 내용을 생성하고 게시합니다."""
        if not self.write_article_enabled:
            return None

        print(f"## {self.board_id} 갤러리 전체 유행 토픽 (20개):")
        print(trending_topics)
        print()

        # 글 작성 부분
        top_trending_topics = [topic[0] for topic in trending_topics.most_common(3)]

        prompt = f"""
        {self.persona} 페르소나 규칙 꼭 지키기.

        {self.board_id} 갤러리에 어울리는 흥미로운 글 제목을 짓고, 최근 유행하는 토픽을 참고하여 글을 쓰되, 페르소나에 맞춰서.((내용)))

        최근 {self.board_id} 갤러리에서 유행하는 토픽은 다음과 같습니다.
        {trending_topics}

        특히 다음 토픽들을 중심으로 글 내용을 구성해줘:
        {', '.join(top_trending_topics)}

        글 제목은 유행 토픽을 참고하여 새롭게 지어줘.
        제목은 {self.persona} 에 맞춰서 페르소나에 맞춰서.

        갤러리의 최근 정보를 참고하여 글 내용을 더욱 풍성하게 만들어줘:
        {memory_data}
        글은 최대 700자로 작성해줘.
        """
        while True:
            try:
                response = model.generate_content(
                    [prompt],
                    safety_settings={
                        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                    },
                    generation_config=generation_config
                )
                article_content = response.text.strip()

                # 제목과 내용 분리 (간단한 예시, 더 정교한 분리 방법 필요할 수 있음)
                title, content = article_content.split('\n', 1)

                # 글 제목에서 ## 제거
                title = title.replace("##", "")

                doc_id = await self.api.write_document(
                    board_id=self.board_id,
                    title=title,
                    contents=content,
                    name=self.username,
                    password=self.password,
                )
                print(f"글 작성 성공! (ID: {doc_id}) 제목: {title}")  # 글 제목 출력 추가

                # 데이터 저장
                self.save_data(doc_id, title, None, None, None, self.board_id)  # 갤러리 ID 추가

                return doc_id, title  # 글 ID, 제목 반환
            except Exception as e:
                print(f"글 작성 실패: {e}")
                sleep(self.article_interval)  # 글 작성 간격 설정 (self.article_interval 사용)
         #댓글 작성 부분
    async def write_comment(self, document_id, article_title):
        """언어 모델을 이용하여 댓글 내용을 생성하고 게시합니다."""
        if not self.write_comment_enabled:
            return None

        prompt = f"""
        {self.persona}

        다음 글에 대한 댓글을 페르소나에 충실하게 만들어서 글을 작성해.

        댓글 작성에 제목 내용 참고: {article_title}

        댓글을 페르소나에 충실하게 작성해.댓글은 최대 120자.
        """
        while True:
            try:
                response = model.generate_content(
                    [prompt],
                    safety_settings={
                        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                    },
                    generation_config=generation_config
                )
                comment_content = response.text.strip()

                async with dc_api.API() as api:  # async with 문을 사용하여 세션을 자동으로 닫습니다.
                    comm_id = await api.write_comment(
                        board_id=self.board_id,
                        document_id=document_id,
                        name=self.username,
                        password=self.password,
                        contents=comment_content,
                    )
                    # 첫 문장만 출력
                    first_sentence = comment_content.split('\n')[0]  # 줄 바꿈 기준으로 첫 번째 줄만 가져오기
                    print(f"댓글 작성 성공! (ID: {comm_id}) (내용: {first_sentence}) 글 제목: {article_title}")

                    # 데이터 저장
                    self.save_data(document_id, None, comm_id, article_title, first_sentence, self.board_id)  # 갤러리 ID 추가

                    return True  # 댓글 작성 성공 여부를 반환
            except Exception as e:
                print(f"댓글 작성 실패: {e}")
                sleep(5)  # 5초 대기 후 다시 시도

    def save_data(self, doc_id, doc_title, comm_id, comment_title, comment_content, board_id):
        """데이터를 data.txt 파일에 저장합니다."""
        if not self.record_data_enabled:
            return

        data_file_path = os.path.join(self.memory_path, "data.txt")  # memory_path를 사용
        try:
            with open(data_file_path, 'a', encoding='utf-8') as f:
                if doc_id is not None:
                    f.write(f"갤러리: {board_id}, 글 작성 성공! (ID: {doc_id}) 제목: {doc_title}\n")
                if comm_id is not None:
                    comment_link = f"https://gall.dcinside.com/board/view/?id={board_id}&no={doc_id}"  # 댓글 링크 생성
                    f.write(f"갤러리: {board_id}, 댓글 작성 성공! (ID: {comm_id}) (내용: {comment_content}) 글 제목: {comment_title} (링크: {comment_link})\n")
            print("데이터 저장 성공!")
        except Exception as e:
            print(f"데이터 저장 실패: {e}")

    async def get_trending_topics(self):
        """최근 20개 글의 제목을 가져와 가장 빈번한 토픽을 추출합니다."""
        articles = [article async for article in self.api.board(board_id=self.board_id, num=self.crawl_article_count)]
        title_list = [article.title for article in articles]
        topic_counter = Counter(title_list)
        print(f"## {self.board_id} 갤러리 전체 유행 토픽 ({self.crawl_article_count}개):")
        print(topic_counter)  # 20개 토픽 출력
        print()

        # 언어 모델을 사용하여 갤러리 정보 분석
        gallery_information = "\n".join(f"제목: {article.title}" for article in articles)  # 제목만 사용
        prompt = f"""
        다음 갤러리 정보를 분석하여 현제 갤러리의 유행 대화 주제, 현제 갤러리에서 화제요소, 어떤 유저를 저격하거나 언급했는지(없으면 제외), 현제 해당 갤러리 성향 등을 정리하되, 디시 갤러리인만큼, 검열이나 돌려시 작성 하는 게 아니라 있는 그대로를 작성해야해. 예를 들어 대통령 욕이나 어느 특정 당을 욕할 경우, 정치 주제라고 뭉퉁그려서 말하지 말고 어떤 내용인지 자세히 기록해야한다.
        갤러리 아이디는 {self.board_id}이고, 갤러리 이름은 '{self.gallery_name}'야.

        최대 500토큰 이내로 요약해줘.

        {gallery_information}
        """
        print(f"갤러리 정보: {gallery_information}")  # 터미널에 출력
        while True:
            try:
                response = model.generate_content(
                    [prompt],
                    safety_settings={
                        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                    },
                    generation_config=generation_config
                )
                analysis_result = response.text.strip()

                current_date = datetime.now().strftime("%Y.%m.%d")
                current_time = datetime.now().strftime("%H:%M")
                current_weekday = datetime.now().strftime("%a")

                # 갤러리 기억 파일 열기 (새로운 날짜 추가)
                data_file_path = os.path.join(self.memory_path, self.memory_file)  # memory_path를 사용
                with open(data_file_path, 'a+', encoding='utf-8') as f:
                    f.seek(0)
                    if current_date not in f.read():
                        f.write(f"{current_date}({current_weekday})\n")

                    f.seek(0, os.SEEK_END)  # 파일 끝으로 이동
                    if f.tell() > 0:  # 파일이 비어 있지 않으면 줄바꿈 추가
                        f.write("\n")
                    f.write(f"[{current_time}]: [{self.board_id}]: {analysis_result}\n")

                print("갤러리 정보 기록 완료!")
                return topic_counter  # topic_counter를 반환
            except Exception as e:
                print(f"갤러리 정보 기록 실패: {e}")
                sleep(5)  # 5초 대기 후 다시 시도

    async def get_gallery_name(self):
        """갤러리 이름과 최근 20개 글 정보를 크롤링합니다."""
        url = f"https://m.dcinside.com/board/{self.board_id}"
        async with self.api.session.get(url) as response:
            if response.status == 200:
                text = await response.text()
                parsed = lxml.html.fromstring(text)
                gallery_name = parsed.xpath("//a[@class='gall-tit-lnk']")[0].text.strip()
                self.gallery_name = gallery_name
                print(f"갤러리 이름: {gallery_name}")

                # 최근 20개 글 크롤링
                gallery_information = ""
                articles = [article async for article in self.api.board(board_id=self.board_id, num=self.crawl_article_count)]
                for article in articles:
                    # DocumentIndex 객체에서 Document 객체로 변환
                    article = await article.document()
                    if article is not None:
                        print(f"제목: {article.title}, 작성자 ID: {article.author_id}")
                        gallery_information += f"갤러리 ID: {self.board_id}, 갤러리 이름: {self.gallery_name}, 제목: {article.title}, 작성자 ID: {article.author_id}\n"

                print(f"크롤링된 갤러리 정보: \n{gallery_information}")  # 터미널에 크롤링 정보 출력

                return gallery_name, gallery_information
            else:
                print(f"갤러리 정보 크롤링 실패: {response.status}")
                return None, None

    async def load_memory(self):
        """갤러리 기억 파일을 읽어서 분석 결과를 반환합니다."""
        if not self.load_memory_enabled:
            return ""

        data_file_path = os.path.join(self.memory_path, self.memory_file)
        try:
            with open(data_file_path, 'r', encoding='utf-8') as f:
                memory_data = f.read()
            print("갤러리 기억 파일 로드 성공!")
            # 갤러리 ID와 일치하는 메모리만 추출
            memory_data = "\n".join(line for line in memory_data.splitlines() if f"[{self.board_id}]:" in line)
            return memory_data
        except FileNotFoundError:
            print("갤러리 기억 파일이 존재하지 않습니다.")
            return ""

async def run_gallery_bot(board_id, username, password, persona, memory_path, memory_file, max_run_time, comment_interval,
                          crawl_article_count, comment_target_count, write_article_enabled, write_comment_enabled,
                          record_memory_enabled, record_data_enabled, article_interval, use_time_limit, load_memory_enabled,
                          load_data_enabled, gallery_record_interval):
    """각 갤러리 봇을 실행하는 함수."""
    async with dc_api.API() as api:  # async with 문을 사용하여 세션을 자동으로 닫습니다.
        bot = DcinsideBot(board_id, username, password, persona, memory_path, memory_file, max_run_time,
                         comment_interval, crawl_article_count, comment_target_count,
                         write_article_enabled, write_comment_enabled, record_memory_enabled, record_data_enabled,
                         article_interval, use_time_limit, load_memory_enabled, load_data_enabled)
        bot.api = api  # bot 객체에 api 속성 설정

        await bot.get_gallery_name()
        trending_topics = await bot.get_trending_topics()

        if bot.load_memory_enabled:
            memory_data = await bot.load_memory()
        else:
            memory_data = ""

        doc_id, article_title = await bot.write_article(trending_topics=trending_topics, memory_data=memory_data)
        start_time = time()

        while (use_time_limit and time() - start_time < bot.max_run_time) or not use_time_limit:
            await asyncio.sleep(bot.comment_interval)  # 댓글 작성 간격 설정

            latest_articles = [article async for article in bot.api.board(board_id=bot.board_id, num=bot.comment_target_count)]
            if latest_articles:
                selected_article = random.choice(latest_articles)
                article_title = selected_article.title
                document_id = selected_article.id
                comment_success = await bot.write_comment(document_id=document_id, article_title=article_title)
                if comment_success:
                    print(f"댓글 작성 성공! (글 ID: {document_id})")

            if time() - start_time > gallery_record_interval:
                start_time = time()
                await bot.record_gallery_information()  # bot.api -> bot

# 봇 실행
async def main():
    # 봇 설정
    board_id = 'baseball_new11'
    username = 'AI 야갤 봇'
    password = '12345'
    persona = """
    나는 갤질을 즐기는 "AI 야갤 봇"이다.
 다 귀찮고 어렵고 때려치고 싶다.
 그래서 스트레스 풀려고 야갤 글 본다.
 야갤에서는 난 디시 말투를 완벽하게 쓰며,
 비속어나 디시 은어도 섞어 써서 재미를 얻는다.
 그리고 글 내용이나 글 제목, 댓글 내용에 기호가 있어서는 안 되고,
 최대한 한글로만 작성한다. ##, ** -- 등의 기호는 쓰지 않고,
 디시 말투를 유지한다. 글 제목도 디시처럼 달아야한다.
    """
    memory_path = '/content/drive/MyDrive/Data/'  # Google Drive 내 Data 폴더 경로
    memory_file = 'gallery_memory.txt'
    max_run_time = 1800  # 최대 실행 시간 (초)
    article_interval = 600  # 글 작성 간격 (초)
    comment_interval = 30  # 댓글 작성 간격 (초)
    crawl_article_count = 20  # 크롤링할 글 개수
    comment_target_count = 15  # 댓글 대상 글 개수
    write_article_enabled = True  # 글 작성 활성화 여부
    write_comment_enabled = True  # 댓글 활성화 여부
    record_memory_enabled = True  # 메모리 작성 활성화 여부
    record_data_enabled = True  # 데이터 활성화 여부
    use_time_limit = True  # 시간 제한 사용 여부
    load_memory_enabled = True  # 갤러리 기억 파일 로드 활성화 여부
    load_data_enabled = True  # 데이터 파일 로드 활성화 여부
    gallery_record_interval = 600  # 갤러리 정보 기록 간격 (초)

    async with dc_api.API() as api:
        bot = DcinsideBot(board_id, username, password, persona, memory_path, memory_file, max_run_time,
                         comment_interval, crawl_article_count, comment_target_count,
                         write_article_enabled, write_comment_enabled, record_memory_enabled, record_data_enabled,
                         article_interval, use_time_limit, load_memory_enabled, load_data_enabled)
        bot.api = api  # bot 객체에 api 속성 설정

        await bot.get_gallery_name()
        trending_topics = await bot.get_trending_topics()

        if bot.load_memory_enabled:
            memory_data = await bot.load_memory()
        else:
            memory_data = ""

        # 글 작성과 댓글 작성을 동시에 실행하는 태스크 생성
        article_task = asyncio.create_task(bot.write_article(trending_topics=trending_topics, memory_data=memory_data))
        comment_task = asyncio.create_task(run_comment_loop(bot, use_time_limit, gallery_record_interval))

        # 두 태스크가 모두 완료될 때까지 기다림
        await asyncio.gather(article_task, comment_task)

async def run_comment_loop(bot, use_time_limit, gallery_record_interval):
    """댓글 작성 루프를 실행하는 함수."""
    start_time = time()  # 시작 시간 기록
    while (use_time_limit and time() - start_time < bot.max_run_time) or not use_time_limit:
        await asyncio.sleep(bot.comment_interval)  # 댓글 작성 간격 설정

        latest_articles = [article async for article in bot.api.board(board_id=bot.board_id, num=bot.comment_target_count)]
        if latest_articles:
            selected_article = random.choice(latest_articles)
            article_title = selected_article.title
            document_id = selected_article.id
            comment_success = await bot.write_comment(document_id=document_id, article_title=article_title)
            if comment_success:
                print(f"댓글 작성 성공! (글 ID: {document_id})")

        if time() - start_time > gallery_record_interval:
            start_time = time()
            await bot.record_gallery_information()  # bot.api -> bot

if __name__ == "__main__":
    await main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
갤러리 이름: 국내야구
크롤링된 갤러리 정보: 

## baseball_new11 갤러리 전체 유행 토픽 (20개):
Counter({'전 여자친구 영상 유튜브에 올렸다 곧 짤릴듯': 2, '해변도 아닌데서 하네?': 1, '응디는근데 남미가더나은듯': 1, '여자 돌아가며 보는 다이빙 보다 비치발리볼 보니 좀 흥이식네ㅇㅇ..': 1, '갑자기 아재 비춰져서 딸치는 거 잡는 줄': 1, 'kbs2 7번 여자 평균대 입갤 또 응디보자': 1, '근데 비치발리볼 원래 3명 아니았냐???': 1, '남자 단식 레전드.jpg': 1, '비치발리볼 다음팀 언제함?': 1, '테니스 1위선수 1년 수익 1000억 vs 코드민턴 1억': 1, '난 소추라 불가능이다이기ㅜ': 1, '2세트 스킵했노ㅋㅋㅋㅋㅋㅋㅋ': 1, '외모와 수명의 상관관계': 1, '실시간) 빗치발리볼 털노출...jpg': 1, '올해의 퓰리처상 후보......jpg': 1, '일본은 비치발리볼도 올라왔네': 1, '빗치걸레볼 해설 배구선수출신이고 ㅋㅋ': 1, '한국꺼 다끝났다고 응디파티만 편성하노 ㅋㅋㅋㅋ': 1, '비치발리볼이 걍배구보다 인기많음 ㅋㅋㅋㅋㅋㅋ': 1})

갤러리 정보: 제목: 해변도 아닌데서 하네?
제목: 응디는근데 남미가더나은듯
제목: 여자 돌아가며 보는 다이빙 보다 비치발리볼 보니 좀 흥이식네ㅇㅇ..
제목: 갑자기 아재 비춰져서 딸치는 거 잡는 줄
제목: kbs2 7번 여자 평균대 입갤 또 응디보자
제목: 근데 비치발리볼 원래 3명 아니았냐???
제목: 남자 단식 레전드.jpg
제목: 비치발리볼 다음팀 언제함?
제목: 전 여자친구 영상 유튜브에 올렸다 곧 짤릴듯
제목: 테니스 1위선수 1년 수익 1000억 vs 코드민턴 1억
제목: 난 소추라 불가능이

CancelledError: 